In [112]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# To Avoid GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [113]:
class Dense(layers.Layer):
    
    def __init__(self, units, input_dim):
        
        super().__init__()
            
        self.w = self.add_weight(
        name = 'w',
        shape = (input_dim, units),
        initializer = 'random_normal',
        trainable = True,
        )
        
        self.b = self.add_weight(
        name = 'b',
        shape = (units, ),
        initializer = 'zero',
        trainable = True,
        )
        
    def call(self, inputs):
            return tf.matmul(inputs, self.w) + self.b


class MyReLu(layers.Layer):
    def __init__(self):
        super().__init__()
        
    def call(self, x):
        return tf.maximum(x, 0)


class MyModel(keras.Model):
    
    def __init__(self, num_classes = 10):
    
        super().__init__()
        
        self.dense1 = Dense(64, 784)
        self.dense2 = Dense(10, 64)
        self.relu = MyReLu()
    
    def call(self, input_tensor):
        
        x = self.relu(self.dense1(input_tensor))
        
        return self.dense2(x)


model = MyModel()

model.compile(
    
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 2s - loss: 0.3465 - accuracy: 0.9042
Epoch 2/10
1875/1875 - 2s - loss: 0.1682 - accuracy: 0.9525
Epoch 3/10
1875/1875 - 2s - loss: 0.1200 - accuracy: 0.9653
Epoch 4/10
1875/1875 - 2s - loss: 0.0938 - accuracy: 0.9720
Epoch 5/10
1875/1875 - 2s - loss: 0.0763 - accuracy: 0.9772
Epoch 6/10
1875/1875 - 2s - loss: 0.0645 - accuracy: 0.9805
Epoch 7/10
1875/1875 - 2s - loss: 0.0542 - accuracy: 0.9831
Epoch 8/10
1875/1875 - 3s - loss: 0.0475 - accuracy: 0.9851
Epoch 9/10
1875/1875 - 3s - loss: 0.0403 - accuracy: 0.9874
Epoch 10/10
1875/1875 - 3s - loss: 0.0358 - accuracy: 0.9890
313/313 - 0s - loss: 0.0869 - accuracy: 0.9739


[0.08688569068908691, 0.9739000201225281]

In [120]:
x_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [52]:
tf.concat([[[1., tf.constant(1.)]], [[0., 1.]]], 0)
tf.constant(1.)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [106]:
class linear_module_up(layers.Layer):
    
    def __init__(self):
        super().__init__()
        self.w = self.add_weight(
        name='w',
        shape=(),
        initializer = 'random_normal',
        trainable = True
        )

    def call(self, x):
        L = tf.concat([[[1., self.w]], [[0., 1.]]], 0)
        return tf.matmul(L, x)

class linear_module_low(layers.Layer):
    
    def __init__(self):
        super().__init__()
        self.w = self.add_weight(
        name='w',
        shape=(),
        initializer = 'random_normal',
        trainable = True
        )

    def call(self, x):
        L = tf.concat([[[1., 0.]], [[self.w, 1.]]], 0)
        return tf.matmul(L, x)



class activation_module_up(layers.Layer):
    
    def __init__(self):
        
        super().__init__()
        
        self.w = self.add_weight(
        name='w',
        shape=(),
        initializer = 'random_normal',
        trainable = True
        )


    def call(self, x):
        
        L = tf.reshape(tf.stack([x[0] + self.w*tf.math.tanh(x[1]), x[1]]),(2,1))
        
        return L

class activation_module_low(layers.Layer):
    
    def __init__(self):
        
        super().__init__()
        
        self.w = self.add_weight(
        name='w',
        shape=(),
        initializer = 'random_normal',
        trainable = True
        )


    def call(self, x):
        
        L = tf.reshape(tf.stack([x[0], x[1] + self.w*tf.math.tanh(x[0])]),(2,1))
        
        return L


In [107]:
class MyModel(keras.Model):
    
    def __init__(self, num_classes = 10):
    
        super().__init__()
        
        self.l1 = linear_module_up()
        self.l2 = linear_module_low()
        
        self.a1 = activation_module_up()
        
        self.l3 = linear_module_up()
        self.l4 = linear_module_low()
        
        self.a2 = activation_module_low()
        
        self.l5 = linear_module_up()
        self.l6 = linear_module_low()
        
        self.a3 = activation_module_up()

    
    def call(self, input_tensor):
        
        boom = self.l1(input_tensor)
        boom1 = self.l2(boom)
        
        boom2 = self.a1(boom1)
        
        boom3 = self.l1(boom2)
        boom4 = self.l2(boom3)
        
        boom5 = self.a2(boom4)
        
        boom6 = self.l1(boom5)
        boom7 = self.l2(boom6)
        
        boom8 = self.a2(boom7)

        return boom8

In [108]:
model = MyModel()

In [109]:
model(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.6980589 ],
       [0.85003537]], dtype=float32)>

In [92]:
l1 = linear_module_up()
l2 = linear_module_low()
a1 = activation_module_up()
a2 = activation_module_low()

In [94]:
x = tf.constant([[1.],[1.]])

boom = l1(x)
boom1 = a1(boom)
boom2 = l2(boom1)
boom3 = a2(boom2)

print(boom3)

tf.Tensor(
[[1.0149446 ]
 [0.95577943]], shape=(2, 1), dtype=float32)
